In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Wed Jun 17 15:50:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 228Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 90.1Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 256Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:03, 161Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 306Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 154Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 117Mit/s]                                                       


In [ ]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
file_name = "elontweets.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.20it/s]


dataset has 87229 tokens
Training...
[10 | 52.80] loss=3.91 avg=3.91
[20 | 97.70] loss=3.49 avg=3.69
[30 | 142.53] loss=3.60 avg=3.66
[40 | 187.37] loss=3.42 avg=3.60
[50 | 232.17] loss=3.43 avg=3.56
[60 | 276.95] loss=3.08 avg=3.48
[70 | 321.71] loss=2.89 avg=3.39
[80 | 366.47] loss=3.02 avg=3.35
[90 | 411.23] loss=2.58 avg=3.26
[100 | 456.00] loss=2.21 avg=3.15
[110 | 500.73] loss=1.95 avg=3.03
[120 | 545.46] loss=2.42 avg=2.98
[130 | 590.21] loss=2.11 avg=2.91
[140 | 634.95] loss=1.57 avg=2.81
[150 | 679.73] loss=1.64 avg=2.72
[160 | 724.46] loss=1.75 avg=2.66
[170 | 769.20] loss=1.58 avg=2.59
[180 | 813.95] loss=1.29 avg=2.51
[190 | 858.68] loss=1.10 avg=2.43
[200 | 903.42] loss=0.91 avg=2.35
======== SAMPLE 1 ========
 it for a second and you should too.
This is the end result of careful calibration and refinement by @elonmusk. Really good @F9 team. Key improvements include: more control over pitch and feel of hexacopter arc
@ID_AA_Carmack pretty much
@ID_AA_Carmack loud and clear

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

My favorite shot of the Falcon 9 launch window from the last flight of the Falcon Heavy booster https://t.co/9EPrgfYQz
RT @SpaceX: Falcon 9 has successfully deployed Falcon Heavy to its target orbit. Launch window opens at 10:32 am EDT, 15:32 UTC. https://t.co/9EPrgfYQz
RT @SpaceX: Successful deployment of #Dragon to its target orbit. Hope you have as well. https://t.co/bn7c3LYaU3
@redletterdave Good point, odds go from 0% to >0% :)
Falcon 9 deployed to full max over Pacific Ocean https://t.co/TbT2kVkZwJ
@oxfordteddy Browser is already a little better. Kernel and browser update in prob 6 weeks or so.
RT @SpaceX: Weather 60% go for #Falcon9 launch today. More ... https://t.co/gtC39uBC7z
@SpaceX_ launch window opens Sunday, 1st of next gen Falcon 9. Weather 60% go for #Dragon test flight. https://t.co/gtC39uBC7z
RT @SpaceX: #Dragon is now vertical in advance of Sunday, 1st launch of the next-generation Falcon 9 https://t.co/gtC39uBC7z https://t.co/j?
@CapnJesus @Lurpatron @MKBHD @jakeraw

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              truncate='https:',
              length=150,
              temperature=0.89,
              prefix="humans",
              nsamples = 5,
              top_k=20,             
              )

humans, it has been a great couple of months ?
RT @SpaceX: More photos from today?s Falcon 9 launch and first stage landing ? https://t.co/095WHWMtKp https://t.co/TpFnWS9GI9
RT @TeslaMotors: Our new Supercharger route in South Australia will soon allow Tesla owners to visit the world's largest battery site https?
@SmileSimplify Fun, exciting tweets coming soon!
@SmileSimplify Sorry about that. You're right. I was depressing myself too :(
@AndrewKemendo Govts don't need to follow normal laws. They will obtain AI developed by companies at gunpoint,
humans: The universe is prob littered w/ the 1-planet graves of cultures which made the sensible economic 
humans.
@Boltfinger Click the left scroll wheel to pause or mute music
@c12elong Yes
@Jbourquat True
@c12elong Drive like crazy if you like loud music
@SpaceX Hyperloop tube vision test footage from earlier today. Not in the works yet.
@ID_AA_Carmack Full performance video of Falcon 9 launch. Not in yet. Early access.
RT @SpaceX: Falcon 9

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=150,
                      temperature=0.7,
                      nsamples=50,
                      batch_size=5
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1